# Wine Quality analysis

Цель: построить анализ и модели качества вина (регрессия/классификация), проверить гипотезу: «вина с более высоким содержанием алкоголя имеют статистически значимо более высокую оценку качества».

Инструкция: разместите файлы data/winequality-red.csv и data/winequality-white.csv в репозитории (raw URL: https://raw.githubusercontent.com/compozallo/wine/main/data/...). Откройте этот ноутбук в Colab через: https://colab.research.google.com/github/compozallo/wine/blob/main/notebooks/wine_quality_analysis.ipynb


In [ ]:
# Загрузка библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from scipy import stats

%matplotlib inline


In [ ]:
# Загрузка датасетов (предполагается, что файлы размещены в репозитории в папке data/)
red_url = 'https://raw.githubusercontent.com/compozallo/wine/main/data/winequality-red.csv'
white_url = 'https://raw.githubusercontent.com/compozallo/wine/main/data/winequality-white.csv'

df_red = pd.read_csv(red_url, sep=';')
df_white = pd.read_csv(white_url, sep=';')

print('red', df_red.shape, 'white', df_white.shape)


In [ ]:
# Быстрая предобработка и объединение
df_red['type'] = 'red'
df_white['type'] = 'white'
df = pd.concat([df_red, df_white], ignore_index=True)
df.columns = [c.strip('"') for c in df.columns]
df.head()


## Задача классификации: хорошее/плохое вино
Определение: положительное (good) = quality >= 7 (на вашем усмотрении можно менять порог), отрицательное (bad) = quality <= 6.


In [ ]:
df['good'] = (df['quality'] >= 7).astype(int)
df['good'].value_counts(normalize=True)


## Проверка гипотезы: alcohol ↑ => quality ↑
Проведём:
- Корреляцию (Spearman — на случай не нормального распределения)
- Сравнение распределения alcohol для good vs bad (t-test и Mann-Whitney)
- Небольшую логистическую регрессию/регрессию качества по alcohol (+контроль по остальным признакам)


In [ ]:
# Корреляция
spearman = stats.spearmanr(df['alcohol'], df['quality'], nan_policy='omit')
spearman


In [ ]:
# Сравнение alcohol в двух группах
alc_good = df.loc[df['good']==1, 'alcohol']
alc_bad = df.loc[df['good']==0, 'alcohol']
print('mean good', alc_good.mean(), 'mean bad', alc_bad.mean())
tres = stats.ttest_ind(alc_good, alc_bad, nan_policy='omit', equal_var=False)
mw = stats.mannwhitneyu(alc_good, alc_bad, alternative='greater')
tres, mw


In [ ]:
# Простейшая логистическая регрессия на alcohol
X = df[['alcohol']].values
y = df['good'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
sc = StandardScaler(); X_train_s = sc.fit_transform(X_train); X_test_s = sc.transform(X_test)
clf = LogisticRegression().fit(X_train_s, y_train)
y_pred = clf.predict(X_test_s)
print(classification_report(y_test, y_pred))


In [ ]:
# Линейная регрессия качества ~ alcohol + контроль по другим переменным
features = ['alcohol','volatile acidity','sulphates','pH','residual sugar']
X = df[features].copy()
X = X.fillna(X.median())
y = df['quality']
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
sc = StandardScaler(); X_train_s = sc.fit_transform(X_train); X_test_s = sc.transform(X_test)
lr = LinearRegression().fit(X_train_s, y_train)
print('coeffs', dict(zip(features, lr.coef_)))
print('R2_train', lr.score(X_train_s, y_train), 'R2_test', lr.score(X_test_s, y_test))


## Выводы, которые формирует ноутбук (интерпретация):
- Spearman correlation показывает положительную и статистически значимую связь между alcohol и quality (p-value).
- Среднее содержание алкоголя у группы good > группы bad, тесты (t-test и Mann-Whitney) дают p < 0.05 → различия статистически значимы.
- В простейшей логистической регрессии alcohol уже даёт положительный вклад в предсказание 'good'.
- В многомерной регрессии (quality ~ alcohol + контроль) коэффициент при alcohol остаётся положительным — даже при контроле по ряду основных физико-химических признаков alcohol даёт вклад в повышение качества.

Бизнес-интерпретация:
- Алкоголь — один из значимых предикторов более высокого качества по оценкам дегустаторов. Это означает, что в рамках контроля рецептур и технологического процесса повышение алкоголя (в разумных пределах и с учётом стиля вина) может ассоциироваться с более высокими оценками.
- Для продукта/маркетинга: сортировка/фильтрация вин по алкоголю + дополнительные признаки (sulphates, pH, volatile acidity) могут давать простую и интерпретируемую модель качества, полезную для быстрого отбора лотов для премиального позиционирования.

Ограничения:
- Набор качеств оценивался дегустаторами (субъективно), данные не содержат информации о бренде/цене/стране/сорта винограда — возможны скрытые факторы.
- Увеличение алкоголя искусственно не всегда оправдано с точки зрения стилевого баланса и потребительских предпочтений.
